## Part 1: Explore the Data

Import the data and use Pandas to learn more about the dataset.

In [19]:
import pandas as pd

df = pd.read_csv('Resources/client_dataset.csv')

df.head()
new_df= df[["first","last","qty","unit_cost","category","order_date","phone","email"]]
new_df

,first,last,qty,unit_cost,category,order_date,phone,email
0,Donald,Harding,105,762.71,decor,2023-04-28,793-904-7725x39308,harding.donald.7185@sullivan.com
1,Tiffany,Myers,21,15.09,consumables,2023-05-19,201.442.4543x942,myers.t.6537@ferguson-johnson.net
2,Shannon,Watson,39,7.86,software,2023-01-29,687.737.9424x8503,swatson8146@payne.net
3,Nathan,Baker,29,24.85,consumables,2023-04-25,827-788-8123x012,bakernathan@benson.com
4,Christina,Schwartz,20,108.17,consumables,2023-05-28,265-829-3643,christinaschwartz9252@mcconnell.com
...,...,...,...,...,...,...,...,...
54634,Lauren,Reese,33,51.60,software,2023-04-26,977.877.5272x11382,late_reese_4081@montoya-chavez.org
54635,Derrick,Moore,47,175.46,consumables,2023-02-08,358.661.5483,derrick.moore.2602@pope.info
54636,Monica,Gutierrez,475,57.31,consumables,2023-03-05,294.805.9100x339,gutierrezm3195@morris.org
54637,Wanda,Solomon,112,1.22,furniture,2023-01-18,(311)767-4924,solomonwanda5962@ross.org


In [16]:
# View the column names in the data
print(df.columns)


Index(['first', 'last', 'job', 'phone', 'email', 'client_id', 'order_id',
       'order_date', 'order_week', 'order_year', 'item_id', 'category',
       'subcategory', 'unit_price', 'unit_cost', 'unit_weight', 'qty',
       'line_number'],
      dtype='object')


In [6]:
# Use the describe function to gather some basic statistics
df = pd.read_csv('data.csv')
df.describe()

In [27]:
# Use this space to do any additional research
# and familiarize yourself with the data.
# Check for missing values
missing_values = df.isnull().sum()
data_types = df.dtypes
summary_stats = df.describe()
sorted_df = df.sort_values(by="last", ascending=False)
sorted_df
sorted_unit_cost = df.sort_values(by="unit_cost", ascending=False)
sorted_unit_cost
sorted_qyt = df.sort_values(by="qty", ascending=False)
df.sort_values(by=["qty", "unit_cost"],ascending=[False,False])

df["subcategory"].value_counts()

subcategory
bathroom supplies     6424
paper products        5874
kitchen supplies      5746
tables                4540
misc                  4286
stands                4201
accounting            3756
project management    3522
plants                3466
wall art              3093
seating               2664
pens                  2603
logistics             1122
telecom                978
audio visual           732
cabling                655
desks                  510
computers              467
Name: count, dtype: int64

In [23]:
# What three item categories had the most entries?
df["category"].value_counts().head(3)






category
consumables    23538
furniture      11915
software        8400
Name: count, dtype: int64

In [24]:
# For the category with the most entries,
# which subcategory had the most entries?
df.loc[df["category"] == "consumables", "subcategory"].value_counts().head(1)


subcategory
bathroom supplies    6424
Name: count, dtype: int64

In [38]:
# Which five clients had the most entries in the data?
df["client_id"].value_counts().head(5)





client_id
33615    220
66037    211
46820    209
38378    207
24741    207
Name: count, dtype: int64

In [33]:
# Store the client ids of those top 5 clients in a list.
top_5_clients = list(df["client_id"].value_counts().head(5).index)
top_5_clients

[33615, 66037, 46820, 38378, 24741]

In [36]:
# How many total units (the qty column) did the
# client with the most entries order order?
df.loc[df["client_id"] == 33615, "qty"].sum()


64313

## Part 2: Transform the Data
Do we know that this client spent the more money than client 66037? If not, how would we find out? Transform the data using the steps below to prepare it for analysis.

In [51]:
# Create a column that calculates the 
# subtotal for each line using the unit_price
# and the qty
df["subtotal"] = df["unit_price"] * df["qty"]
df[["unit_price", "qty", "subtotal"]].head()

df["subtotal"].head(10)



0    115164.00
1       523.95
2       527.28
3      1056.18
4      3902.00
5     13481.52
6     13751.36
7       461.78
8      4109.14
9      5216.64
Name: subtotal, dtype: float64

In [70]:
# Create a column for shipping price.
# Assume a shipping price of $7 per pound
# for orders over 50 pounds and $10 per
# pound for items 50 pounds or under.
def shipping_calc(weight):
    if weight > 50:
        return weight * 7
    return weight * 10

df["total_weight"] = df["qty"] * df["unit_price"]
df["shipping_price"] = df["total_weight"].apply(shipping_calc)
df[["shipping_price", "total_weight", "unit_price" ]].head(3)



,shipping_price,total_weight,unit_price
0,806148.00,115164.00,1096.80
1,3667.65,523.95,24.95
2,3690.96,527.28,13.52


In [73]:
# Create a column for the total price
# using the subtotal and the shipping price
# along with a sales tax of 9.25%

df["line_price"] = round((df["subtotal"] + df["shipping_price"]) * 1.0925, 3)

df["line_price"]



0        1006533.360
1           4579.323
2           4608.427
3           9231.013
4          34103.480
            ...     
54634      23976.355
54635      84863.040
54636     272587.490
54637       1448.742
54638      27122.929
Name: line_price, Length: 54639, dtype: float64

In [77]:
# Create a column for the cost
# of each line using unit cost, qty, and
# shipping price (assume the shipping cost
# is exactly what is charged to the client).

df["cost"] = df["unit_cost"] * df["qty"] + df["shipping_price"]
df.head(5)

,first,last,job,phone,email,client_id,order_id,order_date,order_week,order_year,...,unit_price,unit_cost,unit_weight,qty,line_number,subtotal,total_weight,shipping_price,line_price,cost
0,Donald,Harding,Immunologist,793-904-7725x39308,harding.donald.7185@sullivan.com,58515,8953482,2023-04-28,17,2023,...,1096.80,762.71,7.50,105,1,115164.00,115164.00,806148.00,886232.55,886232.55
1,Tiffany,Myers,Music therapist,201.442.4543x942,myers.t.6537@ferguson-johnson.net,37609,8069089,2023-05-19,20,2023,...,24.95,15.09,1.49,21,0,523.95,523.95,3667.65,3984.54,3984.54
2,Shannon,Watson,Immunologist,687.737.9424x8503,swatson8146@payne.net,57113,1902144,2023-01-29,4,2023,...,13.52,7.86,1.68,39,6,527.28,527.28,3690.96,3997.50,3997.50
3,Nathan,Baker,Accounting technician,827-788-8123x012,bakernathan@benson.com,46554,9031802,2023-04-25,17,2023,...,36.42,24.85,1.23,29,3,1056.18,1056.18,7393.26,8113.91,8113.91
4,Christina,Schwartz,Chiropractor,265-829-3643,christinaschwartz9252@mcconnell.com,92089,1322274,2023-05-28,21,2023,...,195.10,108.17,46.43,20,1,3902.00,3902.00,27314.00,29477.40,29477.40


In [78]:
# Create a column for the profit of
# each line using line cost and line price
df["profit"] = df["line_price"] - df["cost"]

df.sort_values(by="profit", ascending = False)


,first,last,job,phone,email,client_id,order_id,order_date,order_week,order_year,...,unit_cost,unit_weight,qty,line_number,subtotal,total_weight,shipping_price,line_price,cost,profit
0,Donald,Harding,Immunologist,793-904-7725x39308,harding.donald.7185@sullivan.com,58515,8953482,2023-04-28,17,2023,...,762.71,7.50,105,1,115164.00,115164.00,806148.00,886232.55,886232.55,0.0
36418,Brendan,Lopez,Equities trader,532.290.9140,lopezbrendan@white-bailey.com,87985,9504657,2023-05-13,19,2023,...,116.11,10.74,32,2,6707.20,6707.20,46950.40,50665.92,50665.92,0.0
36420,Anna,Molina,Actor,759.220.0827x35845,molina_article@harris.com,84446,1005061,2023-04-08,14,2023,...,40.78,16.57,21,1,907.62,907.62,6353.34,7209.72,7209.72,0.0
36421,Christine,Anderson,"Restaurant manager, fast food",2042205251,christine.anderson.3349@holland.org,77875,5633044,2023-04-06,14,2023,...,54.70,0.89,18,1,1296.36,1296.36,9074.52,10059.12,10059.12,0.0
36422,Mary,Bradley,"Radiographer, diagnostic",760-755-5494x04737,bradleym@perez-williams.com,37608,4208075,2023-05-06,18,2023,...,26.84,4.25,60,1,1799.40,1799.40,12595.80,14206.20,14206.20,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18215,Brendan,Lopez,Equities trader,532.290.9140,lopezbrendan@white-bailey.com,87985,8310375,2023-05-19,20,2023,...,7.99,2.63,119,4,1259.02,1259.02,8813.14,9763.95,9763.95,0.0
18216,Michele,Mcdonald,Emergency planning/management officer,435-931-4939x95137,mcdonaldm317@hess-jackson.com,38473,1544080,2023-02-28,9,2023,...,130.71,4.02,24,5,3781.20,3781.20,26468.40,29605.44,29605.44,0.0
18217,Megan,Powell,Human resources officer,707.285.1167x7011,powell.megan@evans.com,84878,3506329,2023-02-15,7,2023,...,45.17,2.30,25,8,1871.50,1871.50,13100.50,14229.75,14229.75,0.0
18218,Sara,Erickson,Actor,+1-944-446-1134,ericksonsara4512@moreno-mcpherson.com,44881,5613021,2023-05-13,19,2023,...,96.50,3.83,8,6,1026.64,1026.64,7186.48,7958.48,7958.48,0.0


## Part 3: Confirm your work
You have email receipts showing that the total prices for 3 orders. Confirm that your calculations match the receipts. Remember, each order has multiple lines.

Order ID 2742071 had a total price of \$152,811.89

Order ID 2173913 had a total price of \$162,388.71

Order ID 6128929 had a total price of \$923,441.25


In [79]:
# Check your work using the totals above
order_id_list = [ 2742071, 2173913, 6128929]


## Part 4: Summarize and Analyze
Use the new columns with confirmed values to find the following information.

In [16]:
# How much did each of the top 5 clients by quantity
# spend? Check your work from Part 1 for client ids.



33615: $8377308.52
66037: $10259514.79
46820: $9743794.36
38378: $12906550.87
24741: $82268892.02


In [17]:
# Create a summary DataFrame showing the totals for the
# for the top 5 clients with the following information:
# total units purchased, total shipping price,
# total revenue, and total profit. Sort by total profit.



,client_id,qty,shipping_price,line_price,line_cost,line_profit
4,24741,239862,5126448.37,82268892.02,45688899.71,36579992.31
3,38378,73667,3429455.40,12906550.87,9634720.98,3271829.89
1,66037,43018,1395151.85,10259514.79,7004482.98,3255031.81
2,46820,75768,1601448.84,9743794.36,7007191.64,2736602.72
0,33615,64313,1828984.89,8377308.52,6175313.91,2201994.61


In [18]:
# Format the data and rename the columns
# to names suitable for presentation.
# Currency should be in millions of dollars.



In [19]:
# Sort the updated data by "Total Profit" form highest to lowest


,Client ID,Units,Shipping,Total Revenue,Total Cost,Total Profit
4,24741,239862,$5.13M,$82.27M,$45.69M,$36.58M
3,38378,73667,$3.43M,$12.91M,$9.63M,$3.27M
1,66037,43018,$1.40M,$10.26M,$7.00M,$3.26M
2,46820,75768,$1.60M,$9.74M,$7.01M,$2.74M
0,33615,64313,$1.83M,$8.38M,$6.18M,$2.20M
